# Создание Телеграм-ботов. Боты в группах и каналах.

Сегодня мы с вами научимся использовать Телеграм-ботов в группах и каналах. Вдобавок узнаем, как вычленять в сообщениях ссылки, адреса электронных почт и т.д.

---

Ссылки:
* Описание всех методов и классов Telegram Bot API: https://core.telegram.org/bots/api
* Гайд по созданию Телеграм-ботов: https://mastergroosha.github.io/telegram-tutorial/

# Добавляем бота в группу

1. Создаем группу. Для этого перейдите в меню и выберите `Создать группу`.  
2. Добавьте в нее бота как обычного участника.
3. Чтобы бот получил доступ к сообщениям в чате и дополнительным фишкам - кликните по боту в списке участников и выберите `Назначить администратором`.
4. Установите ограничения для вашего бота в качестве админа.

Готово!

# Функции бота в группе

Что же такого может делать бот в группе? Если вы не дадите ему права администратора - бот сможет только отвечать на команды.  

*Например, вы пишете просто сообщение - боты-не администраторы его игнорируют. Пишете `/start`, и эту команду прочитают все боты в группе. Пишете `/start@apo_lectures_bot` - текст с командой `/start@apo_lectures_bot` получит только бот `@apo_lectures_bot`.*


Если же вы сделаете бота администратором - его функционал сильно расширится: он сможет читать все сообщения и отвечать на них.  
Помимо этого, бот может выполнять функции администратора: удалять и добавлять участников, накладывать на них ограничения, удалять системные сообщения, вроде `<имя_пользователя> покинул группу`  


## Ответы на сообщения

Все, что связано с прочтением сообщений и ответом на них - работает так же, как и в обычном чате с ботом. Единственное, хотелось бы отличать сообщения из личного чата с ботом и сообщения из групп.  

Для этого можно посмотреть на поле `message.chat.type`. Это строковое поле. Оно может принимать значения `'private'`, `'group'`, `'supergroup'` или `'channel'`.  

Если нужно работать только с определенной группой - можно записать ее `chat_id`, и отвечать на сообщения только с таким `message.chat.id`.


## Функции администратора

Администраторы групп могут банить участников, убирать их из черного списка, давать участникам разрешения или, наоборот, ограничивать в правах. Это же может делать и бот-администратор.  


### Удаление участника из группы

За бан участника отвечает метод `bot.kick_chat_member(chat_id, user_id, until_date)`.  

`until_date` - это целочисленный аргумент. Он принимает unix-время, в которое юзер будет убран из черного списка. Unix-время - это количество секунд, прошедших с `1 января 1970, 00:00:00 (UTC)`. Получить текущее unix-время можно при помощи функции `time.time()` из модуля `time`.  

Соответственно, чтобы забанить пользователя на минуту - нужно написать: `bot.kick_chat_member(chat_id, user_id, time.time() + 60)`

Если вы устанавливаете бан меньше, чем на `30 секунд` или больше, чем на `366 дней` - то Телеграм считает, что бан установлен навсегда.


### Отмена бана

Отменить блокировку пользователя можно методом `bot.unban_chat_member(chat_id, user_id)`


### Ограничение пользователя в правах

Тут нам поможет метод `bot.restrict_chat_member(chat_id, user_id, until_date)`.  

Помимо этих трех аргументов, у метода есть по аргументу на каждый тип прав. По умолчанию, все они будут ограничены. Но если установить соответствующий аргумент в `True`, то это право останется.

Есть следующие разрешения:
* `can_send_messages`
* `can_send_media_messages`
* `can_send_polls`
* `can_send_other_messages`
* `can_add_web_page_previews`
* `can_change_info`
* `can_invite_users`
* `can_pin_messages`

К сожалению, в библиотеке есть проблемы с этой функцией. Например, запрет на отправку сообщений влечет за собой запрет на отправку медиа, даже если `can_send_media_messages` было `True`. Если вам нужно оставить за пользователем определенные возможности - придется действовать методом проб и ошибок.


### Наделение пользователя правами

Для этого есть метод `bot.promote_chat_member(chat_id, user_id)`.  

Помимо этих трех аргументов, у метода есть по аргументу на каждый тип прав. По умолчанию, все они будут разрешены. Но если установить соответствующий аргумент в `False`, то это право исчезнет.

Есть следующие разрешения:
* `can_change_info`
* `can_post_messages`
* `can_edit_messages`
* `can_delete_messages`
* `can_invite_users`
* `can_restrict_members`
* `can_pin_messages`
* `can_promote_members`

К сожалению, в библиотеке есть проблемы и с этой функцией. Если вам нужно ограничить пользователя в каких-то возможностях - придется действовать методом проб и ошибок.


# Entities

Банить участников или ограничивать их в правах мы хотим в ответ на какие-нибудь действия. Например, если они отправляют в чат спам со ссылками. Но как нам эти ссылки определять? Придется писать регулярные выражения?  

На самом деле, весь специализированный текстовый контент: ссылки, электронные почты, хэштеги - Телеграм умеет помечать сам. Он складывает их в специальное поле `entities` сообщения.  

`entities` - это список из объектов `MessageEntity`. У `MessageEntity` следующие основные поля:

* `type` - основные типы контента: `'mention'`, `'hashtag'`, `'bot_command'`, `'url'`, `'email'`, `'phone_number'`.
* `offset` - индекс символа в `message.text`, с которого начинается контент.
* `length` - длина контента.

Подробнее здесь: https://core.telegram.org/bots/api#messageentity  

Этот код выудит из сообщения пользователя все `entities` и отправит их в чат:

In [ ]:
@bot.message_handler(func=lambda message: message.entities is not None)  # Проверяем, что в сообщении есть entities
def get_entity(message):
    for entity in message.entities:
        text = message.text[entity.offset : entity.offset + entity.length]  # Выбираем кусок сообщения с entity
        bot.send_message(message.chat.id, text)

# Добавляем бота в канал

1. Создаем канал. Для этого перейтите в меню и выберите создать канал.
2. В графе с подписчиками нажмите на кнопку добавления подписчиков, и пригласите вашего бота.


## Как отправлять собщения в канал?

Отправлять сообщения в канал можно так же, как и в чаты. Единственное, во всех функциях вместо `chat_id` нужно указывать строку-название канала.  

Например, если ссылка на ваш канал такая: `https://t.me/repost_channel`, то строка-название канала - `'@repost_channel'`.


## Зачем делать бота администратором канала?

Вот несколько причин:  

* Бот может автоматически пересылать в канал посты из других источников.
* Бот может прикреплять к постам кнопки. Через это можно, например, сделать систему лайков, дизлайков и комментариев.

# Пишем бота-администратора группы

In [ ]:
# Код в проекте PyCharm